In [1]:
!pip install -q pyomo

In [2]:
from pyomo.environ import *

In [3]:
import numpy as np

$\min \ x_1 - 2x_2 + 3x_3\\ \text{ s.t.  } x_1\geq 1, x_2 \geq 2, x_3 \geq 1,\\  2x_1 + x_2 + 2x_3 \leq 10, \\
 -x_1+ x_2 + 2x_3 \leq -3, \\
 2x_1 - 3x_2 + x_3 \leq -6$.

# Exercise 1 part 1

In [4]:
model_lab3 = ConcreteModel()

In [5]:
N = 3 #number of decision variables 
M = 3 #number of constraints without considering the bound constraints  
lb=np.array([1,2,1])
ub=np.array([np.inf,np.inf,np.inf]) 
obj_coef_c = np.array([1,-2,3]) 
constr_coef_A=np.array([[2,1,2],[-1,1,2],[2,-3,1]])
constr_rhs_b=np.array([10,-3,-6])
row_indices=np.arange(M)
print(row_indices)
col_indices = np.arange(N)
print(col_indices)

[0 1 2]
[0 1 2]


In [6]:
model_lab3.x=Var(col_indices)
model_lab3.constraints = ConstraintList()

In [7]:
for i in col_indices:
  model_lab3.x[i].setlb(lb[i])
  model_lab3.x[i].setub(ub[i])


In [8]:
for i in row_indices:
  model_lab3.constraints.add(sum(constr_coef_A[i][j]*model_lab3.x[j] for j in col_indices) <= constr_rhs_b[i])

In [9]:
model_lab3.objective = Objective(expr =summation(obj_coef_c,model_lab3.x), sense=minimize)
model_lab3.pprint() 

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     2 :  None :   inf : False :  True :  Reals
          2 :     1 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] - 2.0*x[1] + 3*x[2]

1 Constraint Declarations
    constraints : Size=3, Index=constraints_index, Active=True
        Key : Lower : Body                   : Upper : Active
          1 :  -Inf : 2*x[0] + x[1] + 2*x[2] :  10.0 

**Solving using glpk solver**

In [10]:
!apt-get install -y -qq glpk-utils

In [11]:
opt = SolverFactory('glpk', executable='/usr/bin/glpsol')
#.solve(model1).write()
result = opt.solve(model_lab3)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 10
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: other
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.011321544647216797

Solver status: ok
Solver termination condition: other


Solving with glpk we are getting status as ok and trmination criterion as other. \\
Here the termination condition is not meaningfull, as from this we cannot say wheather our problem has a solution, it is unfeasible or it has unbounded solutions.

In [12]:
!apt-get install -y -qq coinor-cbc

**Solving using cbc solver**

In [13]:
opt_cbc = SolverFactory('cbc')

In [14]:
result2 = opt_cbc.solve(model_lab3)
print(result2)

print('Solver status:', result2.solver.status)
print('Solver termination condition:',result2.solver.termination_condition)

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>

Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.01545262336730957

Solver status: warning
Solver termination condition: infeasible


Solving with cbc we are getting status as warning and trmination criterion as infeasible. \\
Here the termination condition is meaningfull, as from this we can say that our problem is infeasible.

The reason for the infeasibility is as follows, \\
if we add the constraint 1 and 2 times of constraint 2, we gets $3x_2 + 6x_3 \leq 4$ and then if we add this obtained inequality to the constraint 3, we gets  $2x_1 + 7x_3 \leq -2$ \\
But as $x_1,x_2 \text{ and } x_3 $ are non negative. So, we cannot have any set of values for $x_1,x_2 \text{ and } x_3 $ satisfying the last obtained inequality. \\
Hence, the problem is infeasible.

# Exercise 1 part 2

**(a)**

In [15]:
model_lab3.objective.set_sense(maximize)

In [16]:
model_lab3.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     2 :  None :   inf : False :  True :  Reals
          2 :     1 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : x[0] - 2.0*x[1] + 3*x[2]

1 Constraint Declarations
    constraints : Size=3, Index=constraints_index, Active=True
        Key : Lower : Body                   : Upper : Active
          1 :  -Inf : 2*x[0] + x[1] + 2*x[2] :  10.0 

In [17]:
result3 = opt_cbc.solve(model_lab3)
print(result3)

print('Solver status:', result3.solver.status)
print('Solver termination condition:',result3.solver.termination_condition)

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>

Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: None
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: maximize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.021813631057739258

Solver status: warning
Solver termination condition: infeasible


Solver status: warning \\
Solver termination condition: infeasible \\
Again here also the constraints are all same as before. So, due to the same reason, as explained in Part 1, the problem is infeasible

**(b)**

In [18]:
#returning to original problem
model_lab3.objective.set_sense(minimize)
#setting upper bound
model_lab3.x[1].setub(8)

model_lab3.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     2 :  None :     8 : False :  True :  Reals
          2 :     1 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] - 2.0*x[1] + 3*x[2]

1 Constraint Declarations
    constraints : Size=3, Index=constraints_index, Active=True
        Key : Lower : Body                   : Upper : Active
          1 :  -Inf : 2*x[0] + x[1] + 2*x[2] :  10.0 

In [19]:
result4 = opt_cbc.solve(model_lab3)
print(result4)

print('Solver status:', result4.solver.status)
print('Solver termination condition:',result4.solver.termination_condition)

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>

Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.01
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.026656389236450195

Solver status: warning
Solver termination condition: infeasible


Solver status: warning \\
Solver termination condition: infeasible  \\

Here a new constraint is introduced as upper bound, which do not have any influence on the conclusion we obtained in part 1. So, due to the same reason, as explained in Part 1, the problem is infeasible.

**(c)**

In [20]:
#returning to original problem
model_lab3.x[1].setub(np.inf)
#adding new constraint
new_constr_coef = [0,-1,1]
new_constr_rhs = 9
model_lab3.constraints.add(sum(new_constr_coef[j]*model_lab3.x[j] for j in col_indices) <= new_constr_rhs)

model_lab3.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     2 :  None :   inf : False :  True :  Reals
          2 :     1 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] - 2.0*x[1] + 3*x[2]

1 Constraint Declarations
    constraints : Size=4, Index=constraints_index, Active=True
        Key : Lower : Body                   : Upper : Active
          1 :  -Inf : 2*x[0] + x[1] + 2*x[2] :  10

In [21]:
result5 = opt_cbc.solve(model_lab3)
print(result5)

print('Solver status:', result5.solver.status)
print('Solver termination condition:',result5.solver.termination_condition)

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>

Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.026578426361083984

Solver status: warning
Solver termination condition: infeasible


Solver status: warning \\
Solver termination condition: infeasible  \\

Since before introducing the new constraint, 3 constraints did not had any feasible solution then also after adding a new constraint the set of 4 constraints will not have feasible solution.

**(d)**

In [22]:
#returning to original problem by deactivating the constraint introduced in (c)
model_lab3.constraints[4].deactivate()

In [23]:
#to deactivate constraint 1 and add the new constraint we do the following
model_lab3.constraints[1].deactivate()
model_lab3.constraints.add(expr = model_lab3.x[0]+ model_lab3.x[1] >= 45)

In [24]:
#constructing new objective function
model_lab3.new_objective = Objective(expr = model_lab3.x[0] + model_lab3.x[1]+model_lab3.x[2])

In [25]:
#deactivate the old objective
model_lab3.objective.deactivate()

model_lab3.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     2 :  None :   inf : False :  True :  Reals
          2 :     1 :  None :   inf : False :  True :  Reals

2 Objective Declarations
    new_objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] + x[1] + x[2]
    objective : Size=1, Index=None, Active=False
        Key  : Active : Sense    : Expression
        None :  False : minimize : x[0] - 2.0*x[1] + 3*x[2]

1 Constraint Declarations
    constraints : S

In [26]:
result6 = opt_cbc.solve(model_lab3)
print(result6)

print('Solver status:', result6.solver.status)
print('Solver termination condition:',result6.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 46.0
  Upper bound: 46.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 2
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 2
  Error rc: 0
  Time: 0.020568132400512695
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


Solver status: ok \\
Solver termination condition: optimal \\

Since, when the initial 3 constraints were included together they were causing problem, as we have seen in all above parts.

Here we replaced the 1st constraint with a new one. And the new set of 3 constraints gives feasible solution. hence we are getting optimal solution.

In [27]:
# display solution
print('\nObjective = ', model_lab3.new_objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model_lab3.x[i].value)
print('\nConstraints')
model_lab3.constraints.display()


Objective =  46.0

Decision Variables
x[ 0 ] =  25.6
x[ 1 ] =  19.4
x[ 2 ] =  1.0

Constraints
constraints : Size=5
    Key : Lower : Body               : Upper
      2 :  None : -4.200000000000003 :  -3.0
      3 :  None : -5.999999999999993 :  -6.0
      5 :  45.0 :               45.0 :  None


Here constraints $ 2x_1-3x_2+x_3 \leq -6 \text{ and } x_1+x_2 \geq 45 $  are active